In [48]:
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

In [49]:
from sklearn.preprocessing import LabelEncoder

In [50]:
from torch.nn.utils.rnn import pad_sequence

In [51]:
from gensim.models import Word2Vec

In [52]:
stopwords = stopwords.words('russian')

In [53]:
df = pd.read_csv('./data/Petitions.csv')

In [54]:
print(len(df['reason_category'].unique()))

15


In [55]:
df.drop(columns="id", inplace=True)

In [56]:
new_df = df.sample(100)

In [57]:
rep = new_df["public_petition_text"].to_list()

In [58]:
def remove_html(text): 
    html_tag=re.compile('<.*?>')
    text_no_html = html_tag.sub('', text)
    return text_no_html

In [59]:
def remove_quots(text):
    text_only_letters = re.sub('[^\w\s]', '', text)
    return text_only_letters


In [60]:
def tokenize(text):
    words = word_tokenize(text)
    return words


In [61]:
def morph(text):
    morph = MorphAnalyzer()
    lemmas = [morph.normal_forms(w)[0] for w in text]
    return lemmas

In [62]:
def del_sw(text):
    prepared = [w for w in text if w not in stopwords]
    lemm_sentce = ' '.join(prepared)
    return lemm_sentce

In [63]:
general_mass = {
    "rem_html" : remove_html,
    "rem_quots" : remove_quots,
    "tokenize" : tokenize,
    "morph" : morph,
    "del_sw" : del_sw
}

In [64]:
list_m = ["rem_html", "rem_quots", "tokenize", "morph", "del_sw"]

In [65]:
def prepro(list_m, text):
    for i in list_m:
        text = [general_mass[i](j) for j in text]
    return text

In [66]:
prepro_text = prepro(list_m=list_m, text=rep)

In [67]:
prepro_text

['просить убрать снег желательно убрать машина тротуар наладить механизировать уборка снег',
 'жблом газон',
 'чистить ливневый канализация район дом 21 ул гертовский потоп вода откачивать лично представитель управлять компания ваш сотрудник почистить убрать лёд вдоль бордюр откачать вода первый оттепель снова озеро неисправный сток дорога сливной колодец писать отписка вникнуть проблема просить отремонтировать сток дорога колодец сказать секрет ваш сотрудник это увидеть сток сломать ваш спецтехник который ездить тротуар',
 'грязный лифт',
 'мусор внутридворовый территория 1й парадный газон улица бабушкин дом 63',
 'просить убрать мусор',
 'просить убрать надпись',
 'разбитый покрытие вход подъезд возможно выходить коляска прогулка опасно',
 '4 подъезд подход подъезд люк левый правый сторона провал земля мочь чтоть случиться люк земля разрываться',
 'ненадлежащий состояние малый архитектурный форма придомовый территория требоваться окраскасмфото',
 'убрать',
 'адрес санктпетербург улиц

In [68]:
count_vocab = set((' '.join(prepro_text).split()))
word_to_ix = {word: i for i, word in enumerate(count_vocab)}
word_list = list(word_to_ix.keys())
word_list 

['лист',
 'листва',
 'площадка',
 'заложить',
 'давно',
 'рама',
 'гореть',
 'фекалия',
 'вместо',
 'куча',
 'контейнер',
 'светильник',
 'протяжение',
 'лифт',
 'проверить',
 'актёрский',
 'яма',
 'рябить',
 'спецтехник',
 '4',
 'район',
 'наблюдение',
 'покрытие',
 'бездействовать',
 '63',
 'проспект',
 'опасно',
 'подпрыгивать',
 'продолжать',
 'закрыть',
 'сообщение',
 'жблом',
 'реклама',
 'середина',
 'строительный',
 'разбитый',
 'обеспечить',
 'отписаться',
 'сгрести',
 'самовольный',
 'писать',
 'цветник',
 'xxxxxxxxxxxxx',
 'притвор',
 'элщиток',
 'ужасный',
 'бабушкин',
 'фасад',
 'проём',
 '5',
 'разрываться',
 'отличный',
 'xxxxxxxxxx',
 'асфальтовый',
 'который',
 'повреждение',
 'правый',
 'бела',
 'дорожный',
 'хранение',
 'обращаться',
 'урна',
 '8',
 'заметный',
 'пятница',
 'необходимый',
 'дом',
 'сотрудник',
 'соблюдать',
 '28',
 'решётка',
 'xxxxx',
 'окраскасмфото',
 'администрация',
 'управлять',
 'трансформаторный',
 'открыть',
 'парадный',
 'сугроб',
 'лестнич

In [69]:
AA = list(map(lambda x: list(map(lambda a: word_to_ix[a], x.split(' '))), prepro_text))
word_list_tensor = list(map(lambda x: torch.tensor(x), AA))
word_list_tensor

[tensor([440, 220, 201, 376, 220, 413, 433, 152, 269, 198, 201]),
 tensor([ 31, 451]),
 tensor([452, 136, 443,  20,  66, 121, 373, 253, 174, 131, 283, 437, 238,  74,
         396, 143,  67,  90, 220, 502, 216, 436, 192, 131, 193, 207, 223, 209,
         496, 267, 439, 398, 203,  40, 457, 498, 351, 440, 312, 267, 439, 203,
         497, 243, 143,  67,  89, 333, 267, 461, 143,  18,  54, 327, 433]),
 tensor([310,  13]),
 tensor([316,  84, 428, 150,  77, 451, 349,  46,  66,  24]),
 tensor([440, 220, 316]),
 tensor([440, 220, 472]),
 tensor([ 35,  22, 130, 215, 261, 401, 210, 326,  26]),
 tensor([ 19, 215, 371, 215, 125, 353,  56, 355, 324, 362, 385, 383, 485, 125,
         362,  50]),
 tensor([236, 105, 445, 266, 229, 137, 428, 107,  72]),
 tensor([220]),
 tensor([135, 274, 349, 297,  66,  49, 119, 380,  19, 270,  92, 232,  66]),
 tensor([103, 316]),
 tensor([213, 100, 158, 460, 223, 478, 317]),
 tensor([310, 120, 305, 103, 316]),
 tensor([331, 244, 306]),
 tensor([249,  29, 125, 442, 142,

In [70]:
len(word_list_tensor)

100

In [71]:
w2v = Word2Vec(sentences=word_list, min_count=1, vector_size=32, epochs=50)

In [72]:
w2v.wv.vectors

array([[-0.31056544, -0.28431976,  0.26446304, ..., -0.3733835 ,
        -0.15181595, -0.1568166 ],
       [-0.21347629, -0.3450036 ,  0.25862765, ..., -0.37762025,
        -0.12052578, -0.19990642],
       [-0.28328174, -0.29868165,  0.28039896, ..., -0.3619463 ,
        -0.10685853, -0.12507077],
       ...,
       [ 0.03028108, -0.02904218,  0.02877627, ...,  0.02320009,
        -0.00092264,  0.02302395],
       [-0.28231737, -0.32727784,  0.45453632, ..., -0.34966093,
        -0.20397404, -0.06953479],
       [-0.28342745, -0.3251203 ,  0.42236415, ..., -0.32388267,
        -0.17760906, -0.02318389]], dtype=float32)

In [73]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(new_df["reason_category"])

In [74]:
X_pad = pad_sequence(word_list_tensor, batch_first=True)

In [75]:
y.shape

(100,)

In [76]:
X_pad.size()

torch.Size([100, 55])

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

In [78]:
batch_size = 256
train_ds = TensorDataset(X_train, torch.from_numpy(y_train).type(torch.long))
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

In [79]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [80]:
embed_len = 30
hidden_dim = 30
n_layers=1

In [81]:
import torch.nn.functional as F

In [88]:
class RNNClassifier(nn.Module):
    def __init__(self):
        super(RNNClassifier, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=len(count_vocab), embedding_dim=embed_len)
        self.rnn = nn.RNN(input_size=embed_len, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 15)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        output1, hidden = self.rnn(embeddings, torch.randn(1, len(X_batch), hidden_dim))
        print(hidden.size(), "HIDDEN")
        print(output1.size(), "OUT1")
        output2 = self.linear(output1)
        log_probs = self.softmax(output2)
        return torch.argmax(log_probs, dim=0)

In [89]:
model = RNNClassifier()

In [91]:
model

RNNClassifier(
  (embedding_layer): Embedding(515, 30)
  (rnn): RNN(30, 30, batch_first=True)
  (linear): Linear(in_features=30, out_features=15, bias=True)
  (softmax): LogSoftmax(dim=1)
)

In [85]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [86]:
loss_function = nn.NLLLoss()

In [92]:
epochs = 10

loss_values = []
for epoch in range(epochs):
    for x_b, y_b in train_dl:
        outputs = model(x_b)
        print(outputs.shape)
        loss_value = loss_function(outputs, y_b)

        loss_value.backward()

        optimizer.step()

        optimizer.zero_grad()
    loss_values.append(loss_value.item())

    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')

torch.Size([1, 80, 30]) HIDDEN
torch.Size([80, 55, 30]) OUT1
torch.Size([55, 15])


ValueError: Expected input batch_size (55) to match target batch_size (80).

## LSTM

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self):
        super(LSTMClassifier, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=len(count_vocab), embedding_dim=embed_len)
        self.rnn = nn.LSTM(input_size=embed_len, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 15)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, X_batch):
        
        embeddings = self.embedding_layer(X_batch)
        print(embeddings.size(), "emb")
        output1, hidden = self.rnn(embeddings, torch.randn(1, len(X_batch), hidden_dim))
        print(hidden.size(), "HIDDEN")
        print(output1.size(), "OUT1")
        output2 = self.linear(output1)
        log_probs = self.softmax(output2).mean(1)
        return log_probs

In [ ]:
model2 = LSTMClassifier()

In [ ]:
model2

LSTMClassifier(
  (embedding_layer): Embedding(614, 30)
  (rnn): LSTM(30, 30, batch_first=True)
  (linear): Linear(in_features=30, out_features=15, bias=True)
  (softmax): LogSoftmax(dim=1)
)

In [ ]:
epochs = 10

loss_values2 = []
for epoch in range(epochs):
    for x_b, y_b in train_dl:
        outputs2 = model2(x_b)
        print(outputs2.shape)
        loss_value2 = loss_function(outputs2, y_b)

        loss_value2.backward()

        optimizer.step()

        optimizer.zero_grad()
    loss_values2.append(loss_value2.item())

    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value2.item()}')

torch.Size([80, 102, 30]) emb


IndexError: index 1 is out of bounds for dimension 0 with size 1